In [169]:
import pandas as pd
import numpy as np
import openrouteservice as ors

In [170]:
properties = pd.read_parquet("../data/raw/properties.parquet.gzip")
school = pd.read_csv("../data/raw/SchoolLocation.csv")

In [171]:
school = school.drop(1667)

In [172]:
property_coords = [coord[::-1] for coord in properties['coordinates']]

In [173]:
property_coords = [coord.tolist() for coord in property_coords]

In [174]:
school_coords = school[["X","Y"]].to_numpy().tolist()

In [176]:
client = ors.Client(key='')

In [183]:
len(school_coords)

2298

In [184]:
3500-2298

1202

In [211]:
concat_coords = []
max_property = 3500 - len(school_coords)
remain_length = len(property_coords)
index = 0
while(remain_length > max_property):
    current_concat = property_coords[index*max_property:(index+1)*max_property] + school_coords
    index += 1
    remain_length -= max_property
    concat_coords.append(current_concat)
current_concat = property_coords[len(property_coords) - 103 :] + school_coords
concat_coords.append(current_concat)

In [218]:
distance_matrix_list = []

for coordList in concat_coords:
    school_index = [i for i in range(len(coordList)-len(school_coords), len(coordList))]
    res = client.distance_matrix(
        locations = coordList,
        destinations = school_index,
        metrics = ["distance"],
        units = "km")
    distance_matrix_list.append(res)

ApiError: 400 ({'error': {'code': 6004, 'message': 'Request parameters exceed the server configuration limits. Only a total of 3500 routes are allowed.'}, 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-02-18T19:37:41Z'}, 'timestamp': 1662454555074}})

In [178]:
school_index = [i for i in range(len(property_coords), len(concat_coords))]

12017

In [181]:
res = client.distance_matrix(
    locations = concat_coords,
    destinations = school_index,
    metrics = ["distance"],
    units = "km")

ApiError: 400 ({'error': {'code': 6004, 'message': 'Request parameters exceed the server configuration limits. Only a total of 3500 routes are allowed.'}, 'info': {'engine': {'version': '6.7.0', 'build_date': '2022-02-18T19:37:41Z'}, 'timestamp': 1662453351962}})

In [100]:
res['distances']

[[396.24, 339.19, 213.71],
 [201.52, 364.35, 280.15],
 [0.0, 214.99, 216.15],
 [214.91, 0.0, 122.55],
 [216.59, 123.2, 0.0]]

In [ ]:
m = folium.Map(location=[52.521861, 13.40744], tiles='cartodbpositron', zoom_start=13)

# Some coordinates in Berlin
coordinates = [[13.42731, 52.51088], [13.384116, 52.533558]]

route = client.directions(
    coordinates=coordinates,
    profile='foot-walking',
    format='geojson',
    options={"avoid_features": ["steps"]},
    validate=False,
)
folium.PolyLine(locations=[list(reversed(coord)) 
                           for coord in 
                           route['features'][0]['geometry']['coordinates']]).add_to(m)
    
m

In [7]:
res

{'distances': [[0.0, 5.48], [5.48, 0.0]],
 'destinations': [{'location': [8.344268, 48.233826],
   'snapped_distance': 150.04},
  {'location': [8.343433, 48.263552], 'snapped_distance': 96.58}],
 'sources': [{'location': [8.344268, 48.233826], 'snapped_distance': 150.04},
  {'location': [8.343433, 48.263552], 'snapped_distance': 96.58}],
 'metadata': {'attribution': 'openrouteservice.org | OpenStreetMap contributors',
  'service': 'matrix',
  'timestamp': 1662434600443,
  'query': {'locations': [[8.34234, 48.23424], [8.34423, 48.26424]],
   'profile': 'driving-car',
   'responseType': 'json',
   'metricsStrings': ['DISTANCE'],
   'metrics': ['distance'],
   'units': 'km'},
  'engine': {'version': '6.7.0',
   'build_date': '2022-02-18T19:37:41Z',
   'graph_date': '2022-08-28T14:17:16Z'}}}

In [4]:
coordinates = [[13.384116, 52.533558], [13.428726, 52.519355], [13.41774, 52.498929], [13.374825, 52.496369]]
matrix = client.distance_matrix(
    locations=coordinates,
    profile='foot-walking',
    metrics=['distance', 'duration'],
    validate=False,
)

In [5]:
matrix

{'durations': [[0.0, 2758.8, 3692.7, 3732.83],
  [2758.8, 0.0, 1986.95, 3954.29],
  [3692.7, 1986.95, 0.0, 2510.81],
  [3732.83, 3954.29, 2510.81, 0.0]],
 'distances': [[0.0, 3831.75, 5128.85, 5184.59],
  [3831.75, 0.0, 2759.72, 5492.19],
  [5128.85, 2759.72, 0.0, 3487.3],
  [5184.59, 5492.19, 3487.3, 0.0]],
 'destinations': [{'location': [13.384038, 52.533527],
   'snapped_distance': 6.3},
  {'location': [13.428816, 52.519341], 'snapped_distance': 6.3},
  {'location': [13.417728, 52.498928], 'snapped_distance': 0.83},
  {'location': [13.374358, 52.495524], 'snapped_distance': 99.17}],
 'sources': [{'location': [13.384038, 52.533527], 'snapped_distance': 6.3},
  {'location': [13.428816, 52.519341], 'snapped_distance': 6.3},
  {'location': [13.417728, 52.498928], 'snapped_distance': 0.83},
  {'location': [13.374358, 52.495524], 'snapped_distance': 99.17}],
 'metadata': {'attribution': 'openrouteservice.org | OpenStreetMap contributors',
  'service': 'matrix',
  'timestamp': 166237711053